In [1]:
# MNIST Data Pre-processing
import numpy as np
import keras
import tensorflow as tf
from keras.datasets import mnist
from keras.utils import np_utils  # utilities for one-hot encoding of ground truth values

import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

%load_ext autoreload
%autoreload 2

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
target = 5

x_clean = np.load('../AdvGAN/samples/WB-B-t%d-clean.npy' % target)
x_adv = np.load('../AdvGAN/samples/WB-B-t%d-adv.npy' % target)
x_label = np.load('../AdvGAN/samples/WB-B-t%d-label.npy' % target)

In [3]:
from APEGAN import APEGAN

epochs=500
batch_size=256

N = x_clean.shape[0]
idx = np.random.permutation(N)
x_clean = x_clean[idx]
x_adv = x_adv[idx]
x_label = x_label[idx]

GAN, G, D = APEGAN([28,28,1])

scalarloss = [0,0,0]
for cur_epoch in range(epochs):
    idx = np.random.randint(0, N//5*4, size=batch_size)
    x_clean_batch = x_clean[idx,]
    x_adv_batch = x_adv[idx,]
    scalarloss[0] = D.train_on_batch(x_clean_batch, np.ones(batch_size))/2
    scalarloss[0] += D.train_on_batch(x_adv_batch, np.zeros(batch_size))/2
    GAN.train_on_batch(x_adv_batch, [np.ones(batch_size), x_clean_batch])
    scalarloss[1:] = GAN.train_on_batch(x_adv_batch, [np.ones(batch_size), x_clean_batch])[1:]
    print("Epoch number:",cur_epoch,"; Loss",scalarloss)

/usr/local/lib/python3.5/dist-packages/keras/engine/training.py:975: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Epoch number: 0 ; Loss [4.1377450823783875, 0.015153093, 0.3006407]
Epoch number: 1 ; Loss [0.20905141532421112, 0.045103062, 0.21392475]
Epoch number: 2 ; Loss [0.09153339453041553, 0.084747426, 0.16381787]
Epoch number: 3 ; Loss [0.26408775511663407, 0.08151201, 0.14007124]
Epoch number: 4 ; Loss [0.11763692134991288, 0.04640199, 0.11934894]
Epoch number: 5 ; Loss [0.07091833488084376, 0.032647364, 0.10963754]
Epoch number: 6 ; Loss [0.029901583679020405, 0.009927621, 0.09845484]
Epoch number: 7 ; Loss [0.038613240234553814, 0.0068084737, 0.08938168]
Epoch number: 8 ; Loss [0.035487059503793716, 0.0056649027, 0.08456247]
Epoch number: 9 ; Loss [0.027634032536298037, 0.007869418, 0.080803536]
Epoch number: 10 ; Loss [0.017492716666311026, 0.00718783, 0.077667914]
Epoch number: 11 ; Loss [0.011056960676796734, 0.004770272, 0.0733363]
Epoch number: 12 ; Loss [0.007243614527396858, 0.0067982916, 0.071562074]
Epoch number: 13 ; Loss [0.014958607498556376, 0.009040689, 0.06892879]
Epoch nu

Epoch number: 111 ; Loss [0.0003419819986447692, 0.00067405874, 0.03334064]
Epoch number: 112 ; Loss [0.00034592846350278705, 0.0004895739, 0.031612627]
Epoch number: 113 ; Loss [0.0005030342945246957, 0.0006958124, 0.030960996]
Epoch number: 114 ; Loss [0.0003172287833876908, 0.0003813413, 0.030782048]
Epoch number: 115 ; Loss [0.00039187484071590006, 0.00055811397, 0.031246558]
Epoch number: 116 ; Loss [0.00035999178362544626, 0.0005515397, 0.03064304]
Epoch number: 117 ; Loss [0.00038984215643722564, 0.0007078047, 0.030547755]
Epoch number: 118 ; Loss [0.00035194669908378273, 0.00029068583, 0.03165678]
Epoch number: 119 ; Loss [0.0004096863267477602, 0.0010937102, 0.03139776]
Epoch number: 120 ; Loss [0.00038836633029859513, 0.0004868576, 0.0320106]
Epoch number: 121 ; Loss [0.00034058367600664496, 0.00047762634, 0.03134135]
Epoch number: 122 ; Loss [0.0004722626836155541, 0.0004996411, 0.0318813]
Epoch number: 123 ; Loss [0.00037155199970584363, 0.00080313056, 0.030043283]
Epoch nu

Epoch number: 219 ; Loss [0.00012586532830027863, 0.00039228325, 0.024492025]
Epoch number: 220 ; Loss [0.0001242877115146257, 0.00023121614, 0.02416183]
Epoch number: 221 ; Loss [0.00022339738643495366, 0.00027835873, 0.025114663]
Epoch number: 222 ; Loss [0.00014465626736637205, 0.0004095288, 0.025008252]
Epoch number: 223 ; Loss [0.00014003012984176166, 0.0003395228, 0.025251461]
Epoch number: 224 ; Loss [8.398579666391015e-05, 0.00018121931, 0.024297671]
Epoch number: 225 ; Loss [0.0001056425244314596, 0.00019895393, 0.023827907]
Epoch number: 226 ; Loss [0.00010062888031825423, 0.00021725803, 0.025000568]
Epoch number: 227 ; Loss [0.00015108407751540653, 0.0002261936, 0.024151057]
Epoch number: 228 ; Loss [0.00015015701501397416, 0.00036025565, 0.025258698]
Epoch number: 229 ; Loss [0.0001485150023654569, 0.0002707338, 0.02424676]
Epoch number: 230 ; Loss [0.00019386290659895167, 0.00020504874, 0.02537627]
Epoch number: 231 ; Loss [0.00012478579083108343, 0.00015656496, 0.02421647

Epoch number: 327 ; Loss [6.475666668848135e-05, 6.4508145e-05, 0.02252259]
Epoch number: 328 ; Loss [9.816038800636306e-05, 9.032622e-05, 0.021569805]
Epoch number: 329 ; Loss [8.507693564752117e-05, 0.00010077422, 0.021389224]
Epoch number: 330 ; Loss [4.9265072448179126e-05, 0.00013127238, 0.021537952]
Epoch number: 331 ; Loss [5.296239942254033e-05, 0.00010028971, 0.02160037]
Epoch number: 332 ; Loss [0.000137662747874856, 0.00028138736, 0.02232645]
Epoch number: 333 ; Loss [8.899402746465057e-05, 7.912271e-05, 0.02090418]
Epoch number: 334 ; Loss [0.00016418123777839355, 0.00019355647, 0.021775287]
Epoch number: 335 ; Loss [8.493520363117568e-05, 0.00010986231, 0.022428945]
Epoch number: 336 ; Loss [6.360098723234842e-05, 0.00015707445, 0.022530742]
Epoch number: 337 ; Loss [8.518035701854387e-05, 9.421562e-05, 0.021422425]
Epoch number: 338 ; Loss [8.467747829854488e-05, 0.00033944866, 0.02208201]
Epoch number: 339 ; Loss [0.0001489720252720872, 0.00011873989, 0.021121975]
Epoch 

Epoch number: 435 ; Loss [4.137083305977285e-05, 5.1586787e-05, 0.019976748]
Epoch number: 436 ; Loss [8.396267730859108e-05, 7.9168145e-05, 0.019706888]
Epoch number: 437 ; Loss [3.5499489968060516e-05, 5.809196e-05, 0.01990476]
Epoch number: 438 ; Loss [4.44337183580501e-05, 4.7805377e-05, 0.019783616]
Epoch number: 439 ; Loss [5.88972197874682e-05, 5.4772485e-05, 0.020064186]
Epoch number: 440 ; Loss [3.495821147225797e-05, 4.0468003e-05, 0.020185435]
Epoch number: 441 ; Loss [4.4811932639277074e-05, 0.00021168291, 0.019892346]
Epoch number: 442 ; Loss [7.558946890640073e-05, 7.800707e-05, 0.020370748]
Epoch number: 443 ; Loss [2.5050663680303842e-05, 4.9761133e-05, 0.019289423]
Epoch number: 444 ; Loss [5.61040906177368e-05, 0.00013944964, 0.020294547]
Epoch number: 445 ; Loss [4.43427634309046e-05, 5.4606942e-05, 0.019412106]
Epoch number: 446 ; Loss [3.9758620914653875e-05, 4.7293557e-05, 0.020243797]
Epoch number: 447 ; Loss [7.377104157058056e-05, 7.480016e-05, 0.019484933]
Epo

In [4]:
import matplotlib.pyplot as plt
import pickle

classifier_name = ['A', 'B', 'C']
for cn in classifier_name:
    F = keras.models.load_model('../AdvGAN/models/Classifier-' + cn + '.h5')
    clean = x_clean[N//5*4:]
    adv = x_adv[N//5*4:]
    label = x_label[N//5*4:]
    purified = G.predict(adv)
    adv_pdt = np.argmax(F.predict(adv), axis=1)
    purified_pdt = np.argmax(F.predict(purified), axis=1)
    print('{}, {} : adv acc:{:.4f}, rct acc:{:.4f}'.format(target, cn, np.mean(adv_pdt==label), 
                                                  np.mean(purified_pdt==label)))
    
clean = clean[:100,]
adv = adv[:100,]
purified = purified[:100,]

for k in range(5):
    plt.imshow((clean[k,] * 255).astype(np.int).reshape(28,28))
    plt.show()
    plt.imshow((adv[k,] * 255).astype(np.int).reshape(28,28))
    plt.show()
    plt.imshow((purified[k,] * 255).astype(np.int).reshape(28,28))
    plt.show()

5, A : adv acc:0.0987, rct acc:0.9978


<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

...

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

In [5]:
np.save('./purified_by_APEGAN_trained_on_target_%d.npy' % target, purified)
np.save('./adv_correspond_to_purified_by_APEGAN_trained_on_target_%d.npy' % target, adv)
np.save('./clean_correspond_to_purified_by_APEGAN_trained_on_target_%d.npy' % target, clean)